In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datasets import load_dataset

/Users/hudsonmendes/Workspaces/hudsonmendes/hlm12nli/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ds = load_dataset("snli")

In [4]:
ds

DatasetDict({
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 550152
    })
})

In [5]:
ds["train"].features

{'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None)}

In [6]:
ds["train"][0:3]

{'premise': ['A person on a horse jumps over a broken down airplane.',
  'A person on a horse jumps over a broken down airplane.',
  'A person on a horse jumps over a broken down airplane.'],
 'hypothesis': ['A person is training his horse for a competition.',
  'A person is at a diner, ordering an omelette.',
  'A person is outdoors, on a horse.'],
 'label': [1, 2, 0]}

In [7]:
from typing import List


def split_tokens_and_subtokens(sent: str) -> List[str]:
    tokens = []
    for token in sent.split():
        # collect all chars of token until it finds a non-alphanumeric char
        subtoken = ""
        for char in token:
            if char.isalnum():
                subtoken += char
            else:
                if subtoken:
                    tokens.append(subtoken)
                subtoken = "##" + char
        tokens.append(subtoken)
    return tokens


split_tokens_and_subtokens("A person who's on a horse jumps over a broken down airplane.")

['A',
 'person',
 'who',
 "##'s",
 'on',
 'a',
 'horse',
 'jumps',
 'over',
 'a',
 'broken',
 'down',
 'airplane',
 '##.']

In [8]:
from typing import Optional
import itertools
import random
from tqdm.auto import tqdm
from hlm12nli.tokenisation import Hlm12NliTextTokeniserConfig


def produce_vocab_for(filepath: str, sample_size: Optional[int] = None) -> None:
    ref = Hlm12NliTextTokeniserConfig()
    data = ds["train"]["premise"]
    if sample_size is not None and sample_size > 0:
        data = random.sample(data, sample_size)
    vocab = sorted(set(itertools.chain(*[split_tokens_and_subtokens(sent) for sent in tqdm(data)])))
    vocab = sorted(set([t.lower() for t in vocab]))
    vocab.insert(0, ref.token_end)
    vocab.insert(0, ref.token_str)
    vocab.insert(0, ref.token_oov)
    vocab.insert(0, ref.token_pad)
    with open(filepath, "w") as f:
        f.write("\n".join(vocab))

In [9]:
produce_vocab_for("../data/vocab.txt")

100%|██████████| 550152/550152 [00:10<00:00, 53404.35it/s]


In [10]:
produce_vocab_for("../tests/fixtures/vocab.txt", sample_size=10)

100%|██████████| 10/10 [00:00<00:00, 39199.10it/s]
